In [29]:
import pandas as pd
import os
from bs4 import BeautifulSoup

In [30]:
scores_dir = 'data/scores'

In [31]:
box_scores = os.listdir(scores_dir)
box_scores = [os.path.join(scores_dir, f) for f in box_scores if f.endswith ('.html')]

In [32]:
len(box_scores)

10185

In [33]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    
    if not html.strip():  # check if the HTML is empty or only contains whitespace
        print(f"File {box_score} is empty.")
        return None
    
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [34]:
def read_line_score(soup):
    try: 
        line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
        cols = list(line_score.columns)
        cols[0] = "team"
        cols[-1] = "total"
        line_score.columns = cols
    
        line_score = line_score[["team", "total"]]
        
        return line_score
    
    except ValueError:
        print("No table with id 'line_score' found in HTML.")
        return None

In [35]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [36]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [37]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)
    
    if not soup:
        print(f"Soup is empty for {box_score}. Skipping this game.")
        continue
    
    line_score = read_line_score(soup)
    
    if line_score is None:
        print(f"Skipping game {game} due to error.")
        continue
    
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 10185
200 / 10185
300 / 10185
400 / 10185
500 / 10185
600 / 10185
700 / 10185
800 / 10185
900 / 10185
1000 / 10185
1100 / 10185
1200 / 10185
1300 / 10185
1400 / 10185
1500 / 10185
1600 / 10185
1700 / 10185
1800 / 10185
1900 / 10185
2000 / 10185
2100 / 10185
2200 / 10185
2300 / 10185
2400 / 10185
2500 / 10185
2600 / 10185
2700 / 10185
2800 / 10185
2900 / 10185
3000 / 10185
3100 / 10185
3200 / 10185
3300 / 10185
3400 / 10185
3500 / 10185
3600 / 10185
3700 / 10185
3800 / 10185
3900 / 10185
4000 / 10185
4100 / 10185
4200 / 10185
4300 / 10185
4400 / 10185
4500 / 10185
4600 / 10185
4700 / 10185
4800 / 10185
4900 / 10185
5000 / 10185
5100 / 10185
5200 / 10185
5300 / 10185
5400 / 10185
5500 / 10185
5600 / 10185
5700 / 10185
5800 / 10185
5900 / 10185
6000 / 10185
6100 / 10185
6200 / 10185
6300 / 10185
6400 / 10185
6500 / 10185
6600 / 10185
6700 / 10185
6800 / 10185
6900 / 10185
7000 / 10185
7100 / 10185
7200 / 10185
7300 / 10185
7400 / 10185
7500 / 10185
7600 / 10185
7700 / 10185
7800 / 1

In [38]:
games_df = pd.concat(games, ignore_index=True)
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20357,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
20358,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
20359,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
20360,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [39]:
games_df.to_csv("nba_games.csv")